In [1]:
import numpy as np

from lightonml.projections.sklearn import OPUMap
from lightonopu.opu import OPU
import time 
import os
from sklearn.linear_model import RidgeClassifier
from sklearn import metrics
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from matplotlib import pyplot
import math

import h5py

from tqdm import tqdm_notebook as tqdm

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, TensorDataset

### load data

In [ ]:
#train_random_features = np.load("train_random_features.npy")
#test_random_features = np.load("test_random_features.npy")

#y_train_bin = np.load("labels_train.npy")
#y_test_bin = np.load("labels_test.npy")

In [ ]:
#train_features_tofloat = torch.from_numpy(train_random_features).float()
#train_label_tofloat = torch.from_numpy(y_train_bin).float()

#test_features_tofloat = torch.from_numpy(test_random_features).float()
#test_label_tofloat = torch.from_numpy(y_test_bin).float()

In [2]:
number_of_images_to_use = 4096
number_of_features_to_use = 1600

In [3]:
f_train = h5py.File("../RandomFeatures/3bits4binstrain.h5", 'r')

train_random_features = f_train['inputs'][:number_of_images_to_use,:number_of_features_to_use]
y_train_bin = f_train['labels'][:number_of_images_to_use]
weights_train = f_train['weights'][:number_of_images_to_use]

f_train.close()

In [4]:
f_test = h5py.File("../RandomFeatures/3bits4bintest.h5",'r')

test_random_features = f_test['inputs'][:,:number_of_features_to_use]
y_test_bin = f_test['labels'][:]
weights_test = f_test['weights'][:]

f_test.close()

In [5]:
f_train_raw = h5py.File("../Data/train.h5", 'r')
f_test_raw = h5py.File("../Data/test.h5",'r')

In [6]:
passSR_train = f_train_raw['all_events']['passSR'][:number_of_images_to_use]
passSR_test = f_test_raw['all_events']['passSR'][:]

In [7]:
test_random_features.shape

(137471, 1600)

In [8]:
y_train_bin.shape

(4096,)

### Scale random features

In [9]:
train_random_features = np.sqrt(train_random_features)
test_random_features = np.sqrt(test_random_features)

In [10]:
min_max = preprocessing.MinMaxScaler()
min_max.fit(train_random_features[:10000])

MinMaxScaler(copy=True, feature_range=(0, 1))

In [11]:
for i in tqdm(range(train_random_features.shape[0])):
    train_random_features[i] = min_max.transform(train_random_features[i:i+1])
#train_random_features = min_max.transform(train_random_features)

In [12]:
for i in tqdm(range(test_random_features.shape[0])):
    test_random_features[i] = min_max.transform(test_random_features[i:i+1])
#test_random_features = min_max.transform(test_random_features)

In [ ]:
#train_random_features = np.sqrt(train_random_features)
#test_random_features = np.sqrt(test_random_features)

## Train model

In [13]:
batch_size = 1024
dataset_train = TensorDataset( torch.Tensor(train_random_features), torch.Tensor(y_train_bin),torch.tensor(weights_train))
data_loader_train = DataLoader(dataset_train,batch_size=batch_size,shuffle=True)
#data_loader_test = DataLoader((test_random_features,test_label_tofloat),batch_size=batch_size,shuffle=False)

In [14]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(train_random_features.shape[1], 1)
        

    def forward(self, x):
        x = self.fc1(x)
        #x = torch.sigmoid(x)
        return x

In [15]:
max_weight = torch.max(weights_train)

TypeError: max(): argument 'input' (position 1) must be Tensor, not numpy.ndarray

In [19]:
model = Net()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-5, weight_decay = .1,amsgrad=True)
model.train()

for num_epochs in range(25):
    counter = 0
    for inp, lbl,weight in data_loader_train:
        #print(torch.max(weight))
        logits = model.forward(inp)
        loss = criterion(logits.reshape(logits.shape[0]), lbl)
        loss =(loss * torch.log(weight.float()+1)/(torch.log(weight.float()+1).sum())).sum()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if(counter%10 == 0):
            print('epoch {}, loss {}'.format(num_epochs+1, loss.item()))
        counter = counter+1

epoch 1, loss 0.6978286504745483
epoch 2, loss 0.6574543118476868
epoch 3, loss 0.6356613039970398
epoch 4, loss 0.6314623355865479
epoch 5, loss 0.5842288732528687
epoch 6, loss 0.5759412050247192
epoch 7, loss 0.5432484149932861
epoch 8, loss 0.5800410509109497
epoch 9, loss 0.5170411467552185
epoch 10, loss 0.4774196743965149
epoch 11, loss 0.4913025498390198
epoch 12, loss 0.4632573425769806
epoch 13, loss 0.46513351798057556
epoch 14, loss 0.42309701442718506
epoch 15, loss 0.4065735936164856
epoch 16, loss 0.4417531192302704
epoch 17, loss 0.39135676622390747
epoch 18, loss 0.4088679850101471
epoch 19, loss 0.3863059878349304
epoch 20, loss 0.37081378698349
epoch 21, loss 0.35271477699279785
epoch 22, loss 0.320900559425354
epoch 23, loss 0.3410792946815491
epoch 24, loss 0.336629182100296
epoch 25, loss 0.3130284547805786


In [ ]:
model.eval()
y_output = []
for i in tqdm(range(test_random_features.shape[0])):
    y_output.append(np.array(model(torch.Tensor(test_random_features[i:i+1])).detach()[0]))
    
test_output_sgd = np.array(y_output)

In [ ]:
print(passSR_test.shape)
test_output_sgd.shape

In [ ]:
#np.add(test_output_sgd,passSR_test.reshape(-1,1),test_output_sgd)

In [ ]:
fpr_opu, tpr_opu, thresholds = metrics.roc_curve(y_test_bin,test_output_sgd,sample_weight=weights_test)
metrics.auc(fpr_opu, tpr_opu)

In [ ]:
x_dot = np.sum(weights_test[(passSR_test == 1) & (y_test_bin == 0)]) / np.sum(weights_test[y_test_bin==0])
y_dot = np.sum(weights_test[(passSR_test == 1) & (y_test_bin == 1)]) / np.sum(weights_test[y_test_bin==1])

In [ ]:
pyplot.scatter(x_dot, y_dot,label="Physics selection")
pyplot.plot(fpr_opu, tpr_opu, lw=2,alpha=.6,label="opu",color='red')
pyplot.axis((0,.001,0,1))
pyplot.xlabel("fpr")
pyplot.ylabel("tpr")
pyplot.title("using 16000 OPU Features on entire data")
pyplot.legend(loc='upper right')
#pyplot.savefig("../Results/SHIFTED_OPUOnImages_final_thesis_result_16000_feaures_zoomed_out")

In [ ]:
#np.save("../Results/BDT_24th_feature_test",test_output_sgd)

## Interpret Output

In [ ]:
train_scores = []

for i in range(train_random_features.shape[0]):
    train_scores.append(np.array(model(torch.Tensor(train_random_features[i:i+1])).detach()[0]))
    #print(train_scores[i])

In [ ]:
#np.save("../Results/BDT_24th_feature_train",np.array(train_scores))

In [ ]:
fpr_opu_train, tpr_opu_train, thresholds = metrics.roc_curve(y_train_bin,train_scores,sample_weight=weights_train)
metrics.auc(fpr_opu_train, tpr_opu_train)

In [ ]:
minMax = preprocessing.MinMaxScaler()
minMax.fit(train_scores)
train_scores = minMax.transform(train_scores)
#test_scores = np.array(y_output)
test_output_sgd = minMax.transform(test_output_sgd)

In [ ]:
def compare_train_test(y_pred_train, y_train, y_pred, y_test, high_low=(0,1), bins=30):
    pyplot.hist(y_pred_train[y_train == 1],
                 color='r', alpha=0.5, range=high_low, bins=bins,
                 histtype='stepfilled', density=True,
                 label='S (train)') # alpha is transparancy
    pyplot.hist(y_pred_train[y_train == 0],
                 color='b', alpha=0.5, range=high_low, bins=bins,
                 histtype='stepfilled', density=True,
                 label='B (train)')

    hist, bins = np.histogram(y_pred[y_test == 1], bins=bins, range=high_low, 
                                   density=True)
    scale = len(y_pred[y_test == 1]) / sum(hist)
    err = np.sqrt(hist * scale) / scale

    #width = (bins[1] - bins[0])
    center = (bins[:-1] + bins[1:]) / 2
    pyplot.errorbar(center, hist, yerr=err, fmt='o', c='r', label='S (test)')

    hist, bins = np.histogram(y_pred[y_test == 0],
                                  bins=bins, range=high_low, density=True)
    scale = len(y_pred[y_test == 0]) / sum(hist)
    err = np.sqrt(hist * scale) / scale

    #width = (bins[1] - bins[0])
    center = (bins[:-1] + bins[1:]) / 2
    pyplot.errorbar(center, hist, yerr=err, fmt='o', c='b', label='B (test)')
    pyplot.xlabel("Scores")
    pyplot.ylabel("Arbitrary units")
    pyplot.legend(loc='best')
    
    pyplot.savefig("../Results/scores_plot_FINAL_THESIS_RESULT_16000_Features_3BITS4BINS_LOGofWEIGHT_SMALL-TESTSET")

In [ ]:
compare_train_test(np.array(train_scores),y_train_bin,test_output_sgd,y_test_bin,(0,1))

NameError: name 'plt' is not defined